In [3]:
import os
Path = "F:/02Case studies/Coello"
os.chdir(Path)
#os.listdir(Path)

### Modules

In [5]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import Hapi.hbv_bergestrom92 as HBVLumped

from Hapi.calibration import Calibration
from Hapi.routing import Routing
from Hapi.run import Run
import Hapi.performancecriteria as PC

### Paths

In [6]:
Parameterpath = "Hapi/Model/results/parameters/lumped/Coello_Lumped15062020-1.txt"
Meteopath = "HAPI/Data/00inputs/Lumped Model/"
Path = "HAPI/Data/00inputs/Basic_inputs/"
SaveTo = "Hapi/Model/results/"

### Meteorological data

In [7]:
start = "2009-01-01"
end = "2011-12-31"
name = "Coello"

Coello = Calibration(name, start, end)
Coello.ReadLumpedInputs(Meteopath + "meteo_data-MSWEP.csv")

Lumped Model inputs are read successfully


### Basic_inputs

In [8]:
# catchment area
AreaCoeff = 1530
# temporal resolution
# [Snow pack, Soil moisture, Upper zone, Lower Zone, Water content]
InitialCond = [0,10,10,10,0]
# no snow subroutine
Snow = 0
Coello.ReadLumpedModel(HBVLumped, AreaCoeff, InitialCond)

Lumped model is read successfully


# Calibration parameters

In [12]:
# Calibration boundaries
UB = pd.read_csv(Path + "/lumped/UB-3.txt", index_col = 0, header = None)
parnames = UB.index
UB = UB[1].tolist()
LB = pd.read_csv(Path + "/lumped/LB-3.txt", index_col = 0, header = None)
LB = LB[1].tolist()

Maxbas = True
Coello.ReadParametersBounds(UB, LB, Snow, Maxbas=Maxbas)

# read parameters obtained from previous run to use it as initial values
# parameters = pd.read_csv(Parameterpath, index_col = 0, header = None)
# parameters = parameters[1].tolist()
parameters = []
# Routing
Route=1
RoutingFn = Routing.TriangularRouting1

Basic_inputs=dict(Route=Route, RoutingFn=RoutingFn, InitialValues = parameters)

### Objective function
# outlet discharge
Coello.ReadDischargeGauges(Meteopath+"Qout_c.csv", fmt="%Y-%m-%d")

OF_args=[]
OF=PC.RMSE

Coello.ReadObjectiveFn(PC.RMSE, OF_args)

Parameters bounds are read successfully
Gauges data are read successfully


# Calibration

API options
Create the options dictionary all the optimization parameters should be passed
to the optimization object inside the option dictionary:


to see all options import Optimizer class and check the documentation of the
method setOption

In [13]:
ApiObjArgs = dict(hms=100, hmcr=0.95, par=0.65, dbw=2000, fileout=1, xinit =0,
                      filename=SaveTo + "parameters/01lumped model/Coello_History"+str(dt.datetime.now())[0:10]+".txt")

for i in range(len(ApiObjArgs)):
    print(list(ApiObjArgs.keys())[i], str(ApiObjArgs[list(ApiObjArgs.keys())[i]]))

# pll_type = 'POA'
pll_type = None

ApiSolveArgs = dict(store_sol=True, display_opts=True, store_hst=True, hot_start=False)

OptimizationArgs=[ApiObjArgs, pll_type, ApiSolveArgs]

hms 100
hmcr 0.95
par 0.65
dbw 2000
fileout 1
xinit 0
filename Hapi/Model/results/parameters/01lumped model/Coello_History2021-04-06.txt


### Run Calibration

In [ ]:
cal_parameters = Coello.LumpedCalibration(Basic_inputs, OptimizationArgs, printError=None)

print("Objective Function = " + str(round(cal_parameters[0],2)))
print("Parameters are " + str(cal_parameters[1]))
print("Time = " + str(round(cal_parameters[2]['time']/60,2)) + " min")

Calibration starts

Optimization Problem -- HBV Calibration

        Objective Function: opt_fun

    Objectives:
        Name        Value        Optimum

	Variables (c - continuous, i - integer, d - discrete):
        Name    Type       Value       Lower Bound  Upper Bound
	     x0       c	      0.000000       7.00e-01     1.30e+00 
	     x1       c	      0.000000       5.00e+01     4.00e+02 
	     x2       c	      0.000000       1.15e+00     2.50e+00 
	     x3       c	      0.000000       1.00e-01     3.00e-01 
	     x4       c	      0.000000       4.60e-01     1.00e+00 
	     x5       c	      0.000000       2.27e-01     8.66e-01 
	     x6       c	      0.000000       1.24e-01     8.00e-01 
	     x7       c	      0.000000       5.00e-03     1.07e-01 
	     x8       c	      0.000000       1.61e+01     7.23e+01 
	     x9       c	      0.000000       1.66e+00     5.28e+00 
	    x10       c	      0.000000       1.19e+00     2.00e+00 

46.751776416275774
41.93161832720069
30.595227029888

### Run the Model

In [ ]:
Coello.Parameters = cal_parameters[1]
Run.RunLumped(Coello, Route, RoutingFn)

### Calculate Performance Criteria

In [ ]:
Metrics = dict()

Qobs = Coello.QGauges[Coello.QGauges.columns[0]]

Metrics['RMSE'] = PC.RMSE(Qobs, Coello.Qsim[0])
Metrics['NSE'] = PC.NSE(Qobs, Coello.Qsim[0])
Metrics['NSEhf'] = PC.NSEHF(Qobs, Coello.Qsim[0])
Metrics['KGE'] = PC.KGE(Qobs, Coello.Qsim[0])
Metrics['WB'] = PC.WB(Qobs, Coello.Qsim[0])

print("RMSE= " + str(round(Metrics['RMSE'],2)))
print("NSE= " + str(round(Metrics['NSE'],2)))
print("NSEhf= " + str(round(Metrics['NSEhf'],2)))
print("KGE= " + str(round(Metrics['KGE'],2)))
print("WB= " + str(round(Metrics['WB'],2)))

### Plotting Hydrograph

In [ ]:
plt.figure(1, figsize=(12,8))
plt.plot(Coello.Qsim[0])
plt.plot(Qobs)
plt.xlabel("Time (daily)")
plt.ylabel("Flow Hydrograph m3/s")

### Save the Parameters

In [ ]:
ParPath = SaveTo + "parameters/lumped/Parameters" + str(dt.datetime.now())[0:10] + ".txt"
parameters = pd.DataFrame(index=parnames)
parameters['values'] = cal_parameters[1]
parameters.to_csv(ParPath, header=None, float_format="%0.4f")